In [12]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
load_dotenv()


True

## load variables
- data folder
- cleaned_df
- json template
- 

In [17]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
cleaned_df = pd.read_csv(cleaned_csv)

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)


### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [13]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = '''select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;'''
query2 = '''select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3228,1831,3726,4247,4548,2495);'''
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

#df

## extract halfmoon records for cleaned data
1. get the 'KEY'(works like key and foreign key in the db) values from the main.csv to a list
2. For every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list
3. save the files

In [35]:
#step 1: get the 'KEY' values from cleaned_df to a list
keys = cleaned_df['KEY'].tolist()

#step 2.1: get the other csv filenames
dir = os.path.join(os.getcwd(),'data','brics_csvs')
main_csv = 'Rangeland_Somalia_BRCiSIII.csv'
filenames = [file for file in os.listdir(dir) if file.endswith('.csv') and file != main_csv]

#step2.2 : for every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list

for name in filenames:
    #read the repeat csv
    doi = pd.read_csv(os.path.join(dir,name))

    #construct its new name
    new_filename = f'cleaned_{name}'

    #gey the filtered row values for mtching keys
    filtered_doi = doi[doi['PARENT_KEY'].isin(keys)]

    #save the csvs
    filtered_doi.to_csv(os.path.join(dir,'outputs',new_filename), index=False)

    print(f'💯Created: {new_filename} 🎯with {len(filtered_doi)} records')


💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gully.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv 🎯with 14 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv 🎯with 81 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_keyline.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv 🎯with 26 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terracing.csv 🎯with 6 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv 🎯with 99 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv 🎯with 212 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_basins.csv 🎯with 0 records


## enumerator
get user details from db for json population

In [14]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    i['id'] = idx
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'mrage@so-actionagainsthunger.org',
        'first_name': 'Muktar Mohamednoor',
        'gender': 'Male',
        'id': 1,
        'last_name': 'Rage',
        'organization': 'ACF',
        'phone_number': '252616051599',
        'type': 'ENUMERATOR',
        'username': 'mrage'}
ic| i: {'age_category': '',
        'country': 'Somalia',
        'email': 'khadhiib10@gmail.com',
        'first_name': 'Hassan Omar',
        'gender': 'Male',
        'id': 2,
        'last_name': 'Khadhib',
        'organization': 'Norwegian Refugee Council',
        'phone_number': '0615807566',
        'type': 'ENUMERATOR',
        'username': 'Khadhib'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'suldaanyare1265@gmail.com',
        'first_name': 'Jamaal',
        'gender': 'Male',
        'id': 3,
        'last_name': 'Tagal',
        'organization': 'ACF',
        'phone_number': '2

## farmingEntity
- read column values from cleaned df and append to json keys

In [24]:

#read and set values for farming entity
farming_entity = template[0]['farmingEntity'] 
#ic(farming_entity)

for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    ic(farming_entity)

ic| farming_entity: {'first_name': 'acf',
                     'id': 1,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 2,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 3,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 4,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 5,
                     'last_name': 'acf',
                   

## plot

In [23]:
import uuid
plot_details = template[0]['plot']
for i in cleaned_df.to_dict(orient='records'):
    #ic(i)
    plot_details['name'] = uuid.uuid4().hex
    plot_details['area'] = i['area_ha']
    plot_details['photo_url'] = "https://radrs.icraf.org/uploaded/media/" + i['plot_details-photo']
    ic(plot_details)

ic| plot_details: {'area': 0.469631438,
                   'calculated_size': 6.844758142089844,
                   'conservation_area': True,
                   'crops': ['Tesr'],
                   'farmingEntityId': 1,
                   'fence_type': ['LIVE', 'BARBED'],
                   'fenced': True,
                   'has_crops': True,
                   'id': 1,
                   'livestock_allowed': True,
                   'name': 'a2ee13e08f7f4d139bf88c0187db32a2',
                   'other_fence_type': '',
                   'other_ownership_type': '',
                   'photo_url': 'https://radrs.icraf.org/uploaded/media/1736568888717.jpg',
                   'plot_ownership_type': ['CHURCH_OR_MOSQUE', 'INDIVIDUAL'],
                   'points': [{'accuracy': 0,
                               'altitude': 0,
                               'id': 1,
                               'latitude': -1.270913,
                               'longitude': 36.7781943,
             